In [2]:
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math

In [3]:
df_train=pd.read_csv('data/data_train_with_features.csv')
df_train.head()

,Unnamed: 0,classification,text,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,0,Scientific,the interest in anchoring phenomena and phenom...,['the interest in anchoring phenomena and phen...,222,"['interest', 'anchoring', 'phenomena', 'phenom...",3533,"['interest', 'anchor', 'phenomenon', 'phenomen...","[the, in, and, in, has, been, by, their, in, a...",2426
1,1,Scientific,"galaxy clusters , as the largest peaks in the ...","['galaxy clusters , as the largest peaks in th...",372,"['galaxy', 'clusters', 'largest', 'peaks', 'co...",8982,"['galaxy', 'cluster', 'large', 'peak', 'cosmic...","[as, the, in, the, an, in, and, in, for, and, ...",5768
2,2,Scientific,quantum correlations between components of a s...,['quantum correlations between components of a...,138,"['quantum', 'correlations', 'components', 'sys...",2451,"['quantum', 'correlation', 'component', 'syste...","[between, of, a, or, are, on, the, of, whole, ...",1780
3,3,Scientific,methanol masers are often found in star - form...,['methanol masers are often found in star - fo...,141,"['methanol', 'masers', 'found', 'star', 'formi...",3232,"['methanol', 'maser', 'find', 'star', 'form', ...","[are, often, in, are, two, of, to, i, most, th...",1940
4,4,Scientific,interdisciplinary research has recently gained...,['interdisciplinary research has recently gain...,88,"['interdisciplinary', 'research', 'recently', ...",2695,"['interdisciplinary', 'research', 'recently', ...","[has, a, in, the, of, to, the, in, was, to, tw...",1997


In [4]:
df_train['classification'].unique()

array(['Scientific', 'news', 'reviews', 'story'], dtype=object)

In [5]:
len(df_train[df_train['classification']=='story'].reset_index(drop=True)[450::].text)

50

In [6]:
first_line = df_train['text'].iloc[0]

Zusammenfassung testen

In [7]:
first_line[0:200]

'the interest in anchoring phenomena and phenomena in confined nematic liquid crystals has largely been driven by their potential use in liquid crystal display devices .the twisted nematic liquid cryst'

In [8]:
from transformers import AutoModel,AutoTokenizer

model = AutoModel.from_pretrained('google/pegasus-large')
max_tokens = model.config.max_position_embeddings
print(max_tokens)
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-large')

Some weights of the model checkpoint at google/pegasus-large were not used when initializing PegasusModel: ['final_logits_bias']
- This IS expected if you are initializing PegasusModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing PegasusModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of PegasusModel were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1024


In [9]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer,AutoTokenizer,AutoModelForSeq2SeqLM
import torch
model_name='facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [10]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [11]:
sentences = first_line.split(' .')
splitt=256
# Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
batches = []
batch = []
batch_len = 0
for sentence in sentences:
    sentence_len = len(tokenizer.tokenize(sentence))
    if sentence_len + batch_len > splitt:
        if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 1024 Tokens sind
            batches.append(batch)
            batch = [sentence]
            batch_len = sentence_len
        # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
    else:
        batch.append(sentence)
        batch_len += sentence_len
batches.append(batch)  # fügen Sie den letzten Batch hinzu

In [12]:
batches[0:1][0]
batch_text = '. '.join(batches[0:1][0])

In [13]:
batch_text

'the interest in anchoring phenomena and phenomena in confined nematic liquid crystals has largely been driven by their potential use in liquid crystal display devices. the twisted nematic liquid crystal cell serves as an example. it consists of a nematic liquid crystal confined between two parallel walls , both providing homogeneous planar anchoring but with mutually perpendicular easy directions.  in this casethe orientation of the nematic director is tuned by the application of an external electric or magnetic field. a precise control of the surface alignment extending over large areas is decisive for the functioning of such devices. most studies have focused on nematic liquid crystals in contact with laterally uniform substrates.  on the other hand substrate inhomogeneitiesarise rather naturally as a result of surface treatments such as rubbing. thus the nematic texture near the surface is in fact non - uniform. this non - uniformity , however , is smeared out beyond a decay length

In [14]:
sentences[0:1][0]

'the interest in anchoring phenomena and phenomena in confined nematic liquid crystals has largely been driven by their potential use in liquid crystal display devices'

In [15]:
len(batches[0:1][0])

10

In [16]:

math.floor(len(batch_text.split(' '))*0.8)

180

In [17]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [18]:
def compressionrate(min_rate):
    min_rate=min_rate*0.01
    if min_rate >= 0.5:
        min_rate += 0.1    
    return min_rate, min_rate + 0.1


- 80% :1-0.9
- 70%: 0.9-0.8
- 60% : 0.8-0.7
- 50% : 0.7-0.6
- 40% : 0.55-0.45 --> besonders
- 30% : 0.4-0.3
- 20% : 0.3-0.2

In [19]:
text=summarizer(batch_text,length_penalty=100,num_beams=2)

In [20]:
text[0]['summary_text']

'Nematic liquid crystals are confined between two parallel walls. The orientation of the nematic director is tuned by the application of an external electric or magnetic field. The interest in anchoring phenomena and phenomena in confined nematic liquid. crystals has largely been driven by their potential use in liquid crystal display devices.'

In [21]:
len(text[0]['summary_text'].split(' '))/len(batch_text.split(' '))*100

23.008849557522122

In [22]:
text[0]['summary_text']

'Nematic liquid crystals are confined between two parallel walls. The orientation of the nematic director is tuned by the application of an external electric or magnetic field. The interest in anchoring phenomena and phenomena in confined nematic liquid. crystals has largely been driven by their potential use in liquid crystal display devices.'

In [23]:
max_length_test=math.floor(len(batch_text.split(' '))*max_rate)
min_length_test=math.floor(len(batch_text.split(' '))*min_rate)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2260565325.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2260565325.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'max_rate' is not defined

In [24]:
min_length_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2761637981.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2761637981.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'min_length_test' is not defined

In [25]:
max_length_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3042310743.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3042310743.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'max_length_test' is not defined

In [26]:
text='The interest in anchoring phenomena and phenomena in confined nematic liquid crystals has largely been driven by their potential use in liquid crystal display devices. The orientation of the nematic director is tuned by the application of an external electric or magnetic field. A precise control of the surface alignment extending over large areas is decisive for the functioning of such devices. Most studies have focused on nematic. liquid crystals in contact with laterally uniform substrates.  on the other hand substrate inhomogeneitiesarise rather naturally as a result of surface treatments. such as rubbing. thus the Nematic texture near the surface is in fact non - uniform. very often the thickness of the non- uniform surface layer is considerably smaller than both the wavelength of visible light. and the distance between the two confining parallel walls.'

In [27]:
text[0]['summary_text']

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2637690685.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2637690685.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: string indices must be integers

In [28]:
len(text[0]['summary_text'].split(' '))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3978192947.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3978192947.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: string indices must be integers

In [29]:
from summarizer import Summarizer
bert_model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:

ext_summary = bert_model(batch_text, ratio=0.3)

/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [31]:
len(ext_summary.split(' '))/len(batch_text.split(' '))*100

35.84070796460177

In [32]:
batch = tokenizer(batch_text, truncation=True, padding="longest", return_tensors="pt")
translated = model.generate(**batch,min_length=min_length_test, max_length=max_length_test)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3961870787.py:2 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3961870787.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'min_length_test' is not defined

In [33]:
tgt_text

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/1225410592.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/1225410592.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tgt_text' is not defined

In [34]:
len(tgt_text[0].split(' '))/len(batch_text.split(' '))*100

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3044317567.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3044317567.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tgt_text' is not defined

In [35]:
80*.01

0.8

In [36]:
df_train

,Unnamed: 0,classification,text,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,0,Scientific,the interest in anchoring phenomena and phenom...,['the interest in anchoring phenomena and phen...,222,"['interest', 'anchoring', 'phenomena', 'phenom...",3533,"['interest', 'anchor', 'phenomenon', 'phenomen...","[the, in, and, in, has, been, by, their, in, a...",2426
1,1,Scientific,"galaxy clusters , as the largest peaks in the ...","['galaxy clusters , as the largest peaks in th...",372,"['galaxy', 'clusters', 'largest', 'peaks', 'co...",8982,"['galaxy', 'cluster', 'large', 'peak', 'cosmic...","[as, the, in, the, an, in, and, in, for, and, ...",5768
2,2,Scientific,quantum correlations between components of a s...,['quantum correlations between components of a...,138,"['quantum', 'correlations', 'components', 'sys...",2451,"['quantum', 'correlation', 'component', 'syste...","[between, of, a, or, are, on, the, of, whole, ...",1780
3,3,Scientific,methanol masers are often found in star - form...,['methanol masers are often found in star - fo...,141,"['methanol', 'masers', 'found', 'star', 'formi...",3232,"['methanol', 'maser', 'find', 'star', 'form', ...","[are, often, in, are, two, of, to, i, most, th...",1940
4,4,Scientific,interdisciplinary research has recently gained...,['interdisciplinary research has recently gain...,88,"['interdisciplinary', 'research', 'recently', ...",2695,"['interdisciplinary', 'research', 'recently', ...","[has, a, in, the, of, to, the, in, was, to, tw...",1997
...,...,...,...,...,...,...,...,...,...,...
1995,1995,story,"The room was large, but with a low ceiling, an...","['The room was large, but with a low ceiling, ...",221,"['room', 'large', 'low', 'ceiling', 'end', 'le...",1949,"['room', 'large', 'low', 'ceiling', 'end', 'le...","[The, was, but, with, a, and, at, one, of, the...",2634
1996,1996,story,"""There is nothing more certain than that some ...","['""There is nothing more certain than that som...",98,"['certain', 'day', 'mil', 'record', 'terrible'...",837,"['certain', 'day', 'mil', 'record', 'terrible'...","[There, is, nothing, more, than, that, some, w...",1288
1997,1997,story,In the saloon of the fine Transatlantic liner ...,['In the saloon of the fine Transatlantic line...,121,"['saloon', 'fine', 'Transatlantic', 'liner', '...",865,"['saloon', 'fine', 'transatlantic', 'liner', '...","[In, the, of, the, the, two, from, the, at, on...",1168
1998,1998,story,The trouble about this story is that it really...,['The trouble about this story is that it real...,225,"['trouble', 'story', 'ending', 'Taking', 'ocea...",1725,"['trouble', 'story', 'ending', 'take', 'ocean'...","[The, about, this, is, that, it, really, has, ...",2643


In [37]:
def batch_sent(sentenc,splitt=256,split='. '):
    sentences = first_line.split(split)
    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [38]:
def compressionrate(min_rate):
    min_rate=min_rate*0.01
    if min_rate >= 0.5:
        min_rate += 0.1    
    return min_rate, min_rate + 0.1

In [41]:
df_train[df_train.classification=='Scientific']['text'][0]

"the interest in anchoring phenomena and phenomena in confined nematic liquid crystals has largely been driven by their potential use in liquid crystal display devices .the twisted nematic liquid crystal cell serves as an example .it consists of a nematic liquid crystal confined between two parallel walls , both providing homogeneous planar anchoring but with mutually perpendicular easy directions . in this casethe orientation of the nematic director is tuned by the application of an external electric or magnetic field .a precise control of the surface alignment extending over large areas is decisive for the functioning of such devices .most studies have focused on nematic liquid crystals in contact with laterally uniform substrates . on the other hand substrate inhomogeneitiesarise rather naturally as a result of surface treatments such as rubbing .thus the nematic texture near the surface is in fact non - uniform .this non - uniformity , however , is smeared out beyond a decay length

In [39]:
df_s=df_train[df_train.classification=='news']['text']

In [40]:
df_s[500]

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [146]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
dtype = torch.float
device = torch.device("mps")

True
True


In [43]:
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline


#tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else '[PAD]'
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device = 0)
df_summary_testing = pd.DataFrame(columns=['Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate', 'Zusammenfassung'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3560760537.py:8 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/3560760537.py'                 │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /__init__.py:873 in pipeline                                                                     │
│                                                                                                  │
│   870 │   if device is not None:                                                                 │
│   871 │   │   kwargs["device"] = device                                                          │
│   872 │                                                                                          │
│ ❱ 873 │   return pipeline_class(model=model, framework=framework, task=task, **kwargs)           │
│   874                                                                                            │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /text2text_generation.py:50 in __init__                                                          │
│                                                                                                  │
│    47 │   return_name = "generated"                                                              │
│    48 │                                                                                          │
│    49 │   def __init__(self, *args, **kwargs):                                                   │
│ ❱  50 │   │   super().__init__(*args, **kwargs)                                                  │
│    51 │   │                                                                                      │
│    52 │   │   self.check_model_type(                                                             │
│    53 │   │   │   TF_MODEL_FOR_SEQ_TO_SEQ_CAUSAL_LM_MAPPING                                      │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /base.py:778 in __init__                                                                         │
│                                                                                                  │
│    775 │   │                                                                                     │
│    776 │   │   # Special handling                                                                │
│    777 │   │   if self.framework == "pt" and self.device.type != "cpu":                          │
│ ❱  778 │   │   │   self.model = self.model.to(self.device)                                       │
│    779 │   │                                                                                     │
│    780 │   │   # Update config with task specific parameters                                     │
│    781 │   │   task_specific_params = self.model.config.task_specific_params                     │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/modul │
│ e.py:1132 in to                                                                                  │
│                                                            

In [44]:
counter = 0
# Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
for text in tqdm(df_s):
    compression_rates=random.randint(20, 80)
    compression_rate_min,compression_rate_max = compressionrate(compression_rates)
    text_gesamt_list=[]

    for batch in tqdm(batch_sent(text,split='. '), desc='Verarbeite Batches'):
        # Zusammenfügen der Sätze in einem Batch
        batch_text = '. '.join(batch)
        max_length_test=math.floor(len(batch_text.split(' '))*compression_rate_max)
        min_length_test=math.floor(len(batch_text.split(' '))*compression_rate_min)
        ext_summary=summarizer(batch_text, max_length=max_length_test, min_length=min_length_test,length_penalty=100,num_beams=2)
        # Erstellen Sie einen DataFrame für die aktuellen Ergebnisse
        text_gesamt_list.append(ext_summary[0]['generated_text'])
    text_gesamt = '. '.join(text_gesamt_list)
    actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
    df_current = pd.DataFrame({
        'Min_Kompressionsrate': [compression_rate_min],
        'Max_Kompressionsrate': [compression_rate_max],
        'Endgueltige_Kompressionsrate': [actual_compression_rate],
        'Zusammenfassung': [text_gesamt]
    })

    # Fügen Sie die Daten zum DataFrame hinzu
    df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
    counter += 1

    # Wenn der Zähler ein Vielfaches von 50 ist, speichern Sie die Daten in einer CSV-Datei
    if counter % 50 == 0:
        df_summary_testing.to_csv(f'/content/df_summary_testing_{counter}.csv', index=False)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/500 [00:08<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2170840863.py:13 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_87998/2170840863.py'                 │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /text2text_generation.py:250 in __call__                                                         │
│                                                                                                  │
│   247 │   │   │   - **summary_token_ids** (`torch.Tensor` or `tf.Tensor`, present when `return   │
│   248 │   │   │     ids of the summary.                                                          │
│   249 │   │   """                                                                                │
│ ❱ 250 │   │   return super().__call__(*args, **kwargs)                                           │
│   251 │                                                                                          │
│   252 │   def check_inputs(self, input_length: int, min_length: int, max_length: int) -> bool:   │
│   253 │   │   """                                                                                │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /text2text_generation.py:150 in __call__                                                         │
│                                                                                                  │
│   147 │   │   │     ids of the generated text.                                                   │
│   148 │   │   """                                                                                │
│   149 │   │                                                                                      │
│ ❱ 150 │   │   result = super().__call__(*args, **kwargs)                                         │
│   151 │   │   if (                                                                               │
│   152 │   │   │   isinstance(args[0], list)                                                      │
│   153 │   │   │   and all(isinstance(el, str) for el in args[0])                                 │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /base.py:1074 in __call__                                                                        │
│                                                                                                  │
│   1071 │   │   elif is_iterable:                                                                 │
│   1072 │   │   │   return self.iterate(inputs, preprocess_params, forward_params, postprocess_p  │
│   1073 │   │   else:                                                                             │
│ ❱ 1074 │   │   │   return self.run_single(inputs, preprocess_params, forward_params, postproces  │
│   1075 │                                                                                         │
│   1076 │   def run_multi(self, inputs, preprocess_params, forward_params, postprocess_params):   │
│   1077 │   │   return [self.run_single(item, preprocess_params, forward_params, postprocess_par  │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9

In [ ]:
df_summary_testing.head()